# COSMU File Infector
> Extracting hitchhikers from this 10 year old file infector

- toc: true 
- badges: true
- categories: [cosmu,fileinfector,]

## Overview

This is a file infector that has been propogating for almost ten years. With each propogation it collects a new "infected" file. We are going to write a small extractor script to pull out all the files.

### Samples
- `225715681d8cdf51c5f178e4f4cc67c05608e44cb3d625c108f92caebe4d719b` [UnpacMe](https://www.unpac.me/results/a636d7d6-b98c-431a-8d1d-471f6383ca14?hash=225715681d8cdf51c5f178e4f4cc67c05608e44cb3d625c108f92caebe4d719b)
- `00e0ea6fa8a039786efa9457bbb9b6f13398c256a9bc0eeb71392c2b6657250b` [UnpacMe](https://www.unpac.me/results/f81b66c9-67b1-4e11-aaac-d72a93d0ba7b?hash=00e0ea6fa8a039786efa9457bbb9b6f13398c256a9bc0eeb71392c2b6657250b)

## Analysis
```
_想找回文件-_- _Want to retrieve files-_-
联系作者 恢复所有文件  Contact the author Recover all files
流氓不可怕 就怕流氓有文化  You are not afraid of gangsters. I am afraid that gangsters are educated.
好吧我承认你中奖了  Okay, I admit you won.
不明真相的群众运行了程序  People who don’t know the truth ran the program
```

### Infection Format
```
┌──────────────────────────────────┐                                    
│                                  │                                    
│                                  │                                    
│                                  │                                    
│                                  │ Malware                            
│                                  │                                    
│                                  │                                    
│                                  │                                    
│                                  │                                    
│                                  │                                    
│ ─────────────────────────────────┼───┬───────────                     
│                                  │    Buffer[48] (file name)          
│ ─────────────────────────────────┼───┴───────────                     
│                                  │    Target Size (DWORD)             
│ ─────────────────────────────────┼────────────────                    
│                                  │    Key DWORD (LSB is key)          
├──────────────────────────────────┼────────────────┐                   
│                                  │                │                   
│                                  │                                    
│                                  │                                    
│                                  │                                    
│                                  │                                    
│                                  │  Target                            
│                                  │                                    
│                                  │                                    
│                                  │                                    
│                                  │                                    
│                                  │                                    
│                                  │                                    
│ ─────────────────────────────────┼─────────────────                   
│                                  │   Malware size (DWORD), EOF (DWORD)
└──────────────────────────────────┘                                    
```



In [10]:
import struct
file_name = '/tmp/lol2.bin'
file_data = open(file_name, 'rb').read()

eof_offset = struct.unpack('<I', file_data[-4:])[0]
print(f"EOF: {hex(eof_offset)}")    

malware_size = struct.unpack('<I', file_data[-8:-4])[0]
print(f"Malware Size: {hex(malware_size)}")  

ptr = malware_size
name_data = file_data[ptr:ptr+48]
name = name_data.split(b'\x00')[0].decode()
print(f"Name: {name}")
ptr += 48
target_size = struct.unpack('<I', file_data[ptr:ptr+4])[0]
print(f"Target Size: {hex(target_size)}")
ptr += 4
key_dw = struct.unpack('<I', file_data[ptr:ptr+4])[0]
key = file_data[ptr]
print(f"Key: {hex(key)}")
ptr += 4

target_data = file_data[ptr:ptr+target_size]
open('/tmp/target.bin', 'wb').write(target_data)

out = []
for c in target_data:
    out.append((c - key)&0xff)
open(file_name + '__' + name + '.bin' , 'wb').write(bytes(out))






EOF: 0x2c594
Malware Size: 0x18c1c
Name: bootmgr.exe.mui
Target Size: 0x13938
Key: 0x35


80184

In [19]:
def extract(file_name, file_data):
    try:
        eof_offset = struct.unpack('<I', file_data[-4:])[0]
        malware_size = struct.unpack('<I', file_data[-8:-4])[0]
        ptr = malware_size
        name_data = file_data[ptr:ptr+48]
        name = name_data.split(b'\x00')[0].decode()
        ptr += 48
        target_size = struct.unpack('<I', file_data[ptr:ptr+4])[0]
        ptr += 4
        key_dw = struct.unpack('<I', file_data[ptr:ptr+4])[0]
        key = file_data[ptr]
        ptr += 4
        target_data = file_data[ptr:ptr+target_size]

        if len(name) < 4:
            return 0

        print(f"EOF: {hex(eof_offset)}") 
        print(f"Malware Size: {hex(malware_size)}") 
        print(f"Name: {name}")
        print(f"Target Size: {hex(target_size)}")
        print(f"Key: {hex(key)}")

        out = []
        for c in target_data:
            out.append((c - key)&0xff)
        open(file_name + '__' + name + '.bin' , 'wb').write(bytes(out))
    except Exception as e:
        return 0
    
    return malware_size

# Iterate over all files in directory
import os
for file_name in os.listdir('/tmp/infected2'):
    file_name = '/tmp/infected2/' + file_name
    file_data = open(file_name, 'rb').read()
    print('\n\n')
    print(file_name)
    offset = 1
    while offset > 0:
        offset = extract(file_name, file_data)
        file_data = file_data[:offset]
        print("\n")





/tmp/infected2/7246abf944e0974d921dd9a7d80749383ee387f9583eec20e64837c37cd2d3a8
EOF: 0x763462
Malware Size: 0x207a2
Name: choco.exe
Target Size: 0x742c80
Key: 0x0







/tmp/infected2/1ae200fc20841d638a3bcfb5c1cc71f828f8d4253fcaf76d6047214a607e578d
EOF: 0x1c9588
Malware Size: 0xef7e
Name: utc.privacy.diffbase
Target Size: 0x1ba5ca
Key: 0x2f


EOF: 0xeefd
Malware Size: 0xee3c
Name: desktop.ini
Target Size: 0x81
Key: 0x2f


EOF: 0xecd8
Malware Size: 0xeb34
Name: MS.GROOVE.12.1033.hxn
Target Size: 0x164
Key: 0x33


EOF: 0xe5e2
Malware Size: 0xe050
Name: desktop.ini
Target Size: 0x552
Key: 0x2e


EOF: 0xd72c
Malware Size: 0xcdc8
Name: RefreshEnv.cmd
Target Size: 0x924
Key: 0x32


EOF: 0xcc87
Malware Size: 0xcb06
Name: 10 - UserProfile.lnk
Target Size: 0x141
Key: 0x33







/tmp/infected2/5e036a397730943c7ca0ce35726a8d586659a39f630acd5285465a3e8ee82439
EOF: 0x1fb1b2
Malware Size: 0x40ba8
Name: utc.privacy.diffbase
Target Size: 0x1ba5ca
Key: 0x2e


EOF: 0x807ba
Malware Size: 0x3fbda
Nam